In [ ]:
import requests #library used to make API requests with Python
import json #to read and write json formatted text in Python
import pandas as pd
import os

client_id_google = 'your_client_id_in_string_format'
client_secret_google = 'your_client_secret_in_string_format'
refresh_token_google = 'your_refresh_token_in_string_format'

In [ ]:
#this is the base url to POST a request to be able to get an access 
token
url = "https://www.googleapis.com/oauth2/v4/token"
#note the payload and headers are formatted as a dict
payload = {
    'client_id' : client_id_google ,
    'client_secret' : client_secret_google,
    'refresh_token' : refresh_token_google ,
    'grant_type' : 'refresh_token', 
}
headers = {
    'Content-Type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache",
    }
#the response returns the response code whereas the response.text returns the associated json with the request
response = requests.request("POST", url, data=payload, headers=headers)
#json.loads is used to parse the json object returned with the response
response_dict = json.loads(response.text)
#the values can be accessed like a dict object
google_access_token = response_dict['access_token']

In [ ]:
#dummy dataframe to update sheet with
update_df = pd.DataFrame({'col1': [1,2,3], 'col2': [4,5,6], 'col3': [7,8,10]})

#sheet ID of Google Sheet you want to update
sheetId = 'your_sheet_ID'
#the base Google Sheet API URL is appended with the sheetId and ends with '/values/{range}'
google_put_url = ' https://sheets.googleapis.com/v4/spreadsheets/' + sheetId + '/values/{range}'

#the headers are formatted as a dict with the access token which was retrieved from the previous call adding to the Authorization string. 
#the accept header is not necessary for the write call but required when a read operation is done. This parameter specifies how the response should be formatted. 
google_put_headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer %s' %google_access_token,
  'Accept': 'application/json'
}
#the range parameter of 'Sheet1!A001' specifies that all sheet should be used
google_put_params  ={
  'valueInputOption': 'USER_ENTERED',
    "range": "Sheet1!A001",
    }

#the dataframe that will be used to update the sheet needs to be formatted correctly for Google Sheets to receive it as input
result = [update_df.columns.values.tolist()]
result.extend(update_df.values.tolist())
#it is crucial to convert the dataframe into a dictionary as shown below and to use json.dumps so that the request body can received by the api. There will be an error if json.dumps() is not used and the input is manually formatted into a json. 
request_body = json.dumps(  {"majorDimension": "ROWS","values": result})

#to update the google sheet a PUT request needs to be used
google_response = requests.request("PUT", google_put_url, headers=google_put_headers,params= google_put_params, data=request_body )
google_response_pretty_json = json.loads(google_response.text)

#this print statement formats the response nicely she if the operation has been successful returns the sheet ID and update details
print(json.dumps(google_response_pretty_json, indent=4))